<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# 模块 2.2: 组合逻辑
**上一节: [你的第一个 Chisel 模块](2.1_first_module_zh.ipynb)**<br>
**下一节: [控制流](2.3_control_flow_zh.ipynb)**

## 动机

在本节中，您将看到如何使用Chisel组件来实现组合逻辑。

我们将演示三种基本的 Chisel 类型：`UInt` -- 无符号整数；`SInt` -- 有符号整数；`Bool` -- true 或 false 如何被连接和操作。

注意所有的 Chisel 变量都声明为 Scala 的 `val`。

永远不要将 Scala 的 `var`用于硬件构造体，因为构造体本身一旦被定义就永远不会改变，只有它的值在运行硬件时才会改变。

线路可以用参数化的类型。

## 设置

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/chisel-bootcamp/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

---
# 常见的运算符

现在你已经了解了 `Module` 是如何构造的，让我们来实现一些硬件吧! 看看下面的空模块。

In [3]:
class MyModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
}

defined class MyModule

我们把我们的类叫做 `MyModule`，它基于 `Module`，这意味着它在 Verilog 中被映射到一个硬件模块。我们的 `MyModule` 模块有一个输入和一个输出。输入是一个4位无符号整数 (`UInt`)，输出也是如此。

<span style="color:blue">**示例: Scala 和 Chisel 运算符看起来一样**</span><br>

让我们看看可以对数据进行的不同操作。

In [4]:
class MyModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })

  val two  = 1 + 1
  println(two)
  val utwo = 1.U + 1.U
  println(utwo)
  
  io.out := io.in
}
println(getVerilog(new MyModule))

Elaborating design...
2
UInt<1>(OpResult in MyModule)
Done elaborating.
module MyModule(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out
);
  assign io_out = io_in; // @[cmd3.sc 12:10]
endmodule



defined class MyModule

我们创建了两个 `val`。第一个将两个 Scala 的 `Int` 加起来，所以 `println` 打印出了整数 2。第二个 `val` 将两个 *Chisel* 的 `UInt` 加在一起，所以 `println` 将其视为一个硬件节点并打印出类型名称和指针（`chisel3.core.UInt@d`）。注意，`1.U` 是一个从 Scala 的 `Int` (1) 到 Chisel 的 `UInt` 的字面类型转换。

我们需要驱动输出的东西，所以我们现在只是把它和输入连接起来，就像前面的教程中的直通模块一样。

<span style="color:blue">**示例：不兼容的操作**</span>

如果我们在字面 `1` 上加一个 Chisel `1.U` 会发生什么？这些类型是不兼容的，因为后者是一个值为 1 的硬件线，而前者是一个值为 1 的 Scala，所以 Chisel 会给出一个类型不匹配的错误。

In [5]:
class MyModuleTwo extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })

  val twotwo = 1.U + 1
  println(twotwo)
  
  io.out := io.in
}
println(getVerilog(new MyModuleTwo))

cmd4.sc:7: type mismatch;
 found   : Int(1)
 required: chisel3.UInt
  val twotwo = 1.U + 1
                     ^Compilation Failed

: 

在执行操作时，记住类型之间的区别是很重要的。Scala 是一种强类型的语言，所以任何类型的转换都必须是显式的。

<span style="color:blue">**示例：更多 Chisel 操作符**</span>

其他常见的操作是减法和乘法。这些都是对无符号整数的处理，正如预期的那样。让我们看看这些操作的效果。我们展示的是 Verilog，尽管有一些底层的 Chisel 特性混淆了我们期望的简单代码。

In [5]:
class MyOperators extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(4.W))
    val out_add = Output(UInt(4.W))
    val out_sub = Output(UInt(4.W))
    val out_mul = Output(UInt(4.W))
  })

  io.out_add := 1.U + 4.U
  io.out_sub := 2.U - 1.U
  io.out_mul := 4.U * 2.U
}
println(getVerilog(new MyOperators))

Elaborating design...
Done elaborating.
module MyOperators(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out_add,
  output [3:0] io_out_sub,
  output [3:0] io_out_mul
);
  wire [1:0] _T_3 = 2'h2 - 2'h1; // @[cmd4.sc 10:21]
  wire [4:0] _T_4 = 3'h4 * 2'h2; // @[cmd4.sc 11:21]
  assign io_out_add = 4'h5; // @[cmd4.sc 9:21]
  assign io_out_sub = {{2'd0}, _T_3}; // @[cmd4.sc 10:21]
  assign io_out_mul = _T_4[3:0]; // @[cmd4.sc 11:14]
endmodule



defined class MyOperators

而这里是一个用于上述操作的测试。我们将创建一个显式的测试类，而不是像上一个教程中那样使用一个匿名的测试类。这只是编写测试的另一种方法。

In [6]:
test(new MyOperators) {c =>
  c.io.out_add.expect(5.U)
  c.io.out_sub.expect(1.U)
  c.io.out_mul.expect(8.U)
}
println("SUCCESS!!")

Elaborating design...
Done elaborating.
test MyOperators Success: 0 tests passed in 2 cycles in 0.064905 seconds 30.81 Hz
SUCCESS!!


<span style="color:blue">**示例：Mux和Concatenation**</span>

除了加法、减法和乘法之外，Chisel 还有 mux 和 concatenation 运算符。这些运算符如下所示。`Mux` 的操作类似于传统的三元运算符，其顺序是（条件，如果为真，如果为假）。注意，`true.B` 和 `false.B` 是创建 Chisel Bool 字面的首选方式。`Cat` 顺序是先 MSB 后 LSB（其中 B 指的是位或比特），并且只需要两个参数。连接两个以上的值需要多次调用 `Cat` 或后面章节中涉及的 Chisel 和 Scala 高级功能。


In [7]:
class MyOperatorsTwo extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(4.W))
    val out_mux = Output(UInt(4.W))
    val out_cat = Output(UInt(4.W))
  })

  val s = true.B
  io.out_mux := Mux(s, 3.U, 0.U) // should return 3.U, since s is true
  io.out_cat := Cat(2.U, 1.U)    // concatenates 2 (b10) with 1 (b1) to give 5 (101)
}

println(getVerilog(new MyOperatorsTwo))

test(new MyOperatorsTwo) { c =>
  c.io.out_mux.expect(3.U)
  c.io.out_cat.expect(5.U)
}
println("SUCCESS!!")

Elaborating design...
Done elaborating.
module MyOperatorsTwo(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out_mux,
  output [3:0] io_out_cat
);
  assign io_out_mux = 4'h3; // @[cmd6.sc 9:20]
  assign io_out_cat = 4'h5; // @[Cat.scala 30:58]
endmodule

Elaborating design...
Done elaborating.
test MyOperatorsTwo Success: 0 tests passed in 2 cycles in 0.004082 seconds 489.98 Hz
SUCCESS!!


defined class MyOperatorsTwo

请注意 Verilog 使用常数而不是实际的多路复用或连接逻辑。这是因为 FIRRTL 转换已经简化了电路，消除了明显的逻辑。

关于 Chisel 运算符的更完整的列表，请参阅 [Chisel cheatsheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf)。对于最完整的运算符列表和它们的实现细节，请查看 [Chisel API](https://chisel-lang.org/api/latest/)。

---
# Exercises
To complete these exercises, you may need to look through the [Chisel cheatsheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf).

<span style="color:red">**Exercise: MAC**</span><br>
Create a Chisel module that implements the multiply accumulate function, `(A*B)+C`, and passes the testbench.

In [ ]:
class MAC extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val in_c = Input(UInt(4.W))
    val out  = Output(UInt(8.W))
  })

  ???
}

test(new MAC) { c =>
  val cycles = 100
  import scala.util.Random
  for (i <- 0 until cycles) {
    val in_a = Random.nextInt(16)
    val in_b = Random.nextInt(16)
    val in_c = Random.nextInt(16)
    c.io.in_a.poke(in_a.U)
    c.io.in_b.poke(in_b.U)
    c.io.in_c.poke(in_c.U)
    c.io.out.expect((in_a * in_b + in_c).U)
  }
}
println("SUCCESS!!")

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
class MAC extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val in_c = Input(UInt(4.W))
    val out  = Output(UInt(8.W))
  })

  io.out := (io.in_a * io.in_b) + io.in_c
}
</pre></article></div></section></div>

<span style="color:red">**Exercise: Arbiter**</span><br>
The following circuit arbitrates data coming from a FIFO into two parallel processing units. The FIFO and processing elements (PEs) communicate with ready-valid interfaces. Construct the arbiter to send data to whichever PE is ready to receive data, prioritizing PE0 if both are ready to receive data. Remember that the arbiter should tell the FIFO that it's ready to receive data when at least one of the PEs can receive data. Also, wait for a PE to assert that it's ready before asserting that the data are valid. You will likely need binary operators to complete this exercise.

<img src="images/arbiter.png" width="687" height="177">

In [ ]:
class Arbiter extends Module {
  val io = IO(new Bundle {
    // FIFO
    val fifo_valid = Input(Bool())
    val fifo_ready = Output(Bool())
    val fifo_data  = Input(UInt(16.W))
    
    // PE0
    val pe0_valid  = Output(Bool())
    val pe0_ready  = Input(Bool())
    val pe0_data   = Output(UInt(16.W))
    
    // PE1
    val pe1_valid  = Output(Bool())
    val pe1_ready  = Input(Bool())
    val pe1_data   = Output(UInt(16.W))
  })

  ???
}

test(new Arbiter) { c =>
  import scala.util.Random
  val data = Random.nextInt(65536)
  c.io.fifo_data.poke(data.U)
  
  for (i <- 0 until 8) {
    c.io.fifo_valid.poke((((i >> 0) % 2) != 0).B)
    c.io.pe0_ready.poke((((i >> 1) % 2) != 0).B)
    c.io.pe1_ready.poke((((i >> 2) % 2) != 0).B)

    c.io.fifo_ready.expect((i > 1).B)
    c.io.pe0_valid.expect((i == 3 || i == 7).B)
    c.io.pe1_valid.expect((i == 5).B)
    
    if (i == 3 || i ==7) {
      c.io.pe0_data.expect((data).U)
    } else if (i == 5) {
      c.io.pe1_data.expect((data).U)
    }
  }
}
println("SUCCESS!!")

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  io.fifo_ready := io.pe0_ready || io.pe1_ready
  io.pe0_valid := io.fifo_valid && io.pe0_ready
  io.pe1_valid := io.fifo_valid && io.pe1_ready && !io.pe0_ready
  io.pe0_data := io.fifo_data
  io.pe1_data := io.fifo_data
</pre></article></div></section></div>

<span style="color:red">**Exercise: Parameterized Adder (Optional)**</span><br>
This optional exercise exposes you to one of the most powerful features of Chisel, it's parameterization capabilities. To demonstrate this, we'll construct a parameterized adder that can either saturate the output when overflow occurs, or truncate the results (i.e. wrap around).

First, look at the `Module` below. The parameter we pass into it is called `saturate` and has type *Scala* `Boolean`. This is not a Chisel `Bool`. So, we're not creating a single hardware adder that can either saturate or truncate, but rather we're creating a *generator* that produces either a saturating hardware adder *or* a truncating hardware adder. The decision is made at compile time.

Next, notice the inputs and outputs are all 4-bit `UInt`s. Chisel has built-in width inferencing, and if you look at the [cheatsheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf), you'll see that the bitwidth of a normal summation is equal to the maximum bitwidth of the two inputs. This means that

```scala
val sum = io.in_a + io.in_b
```

will make `sum` a 4-bit wire, and the value will be the truncated result for 4-bit inputs. To check if the summation should saturate, you need to place the result in a 5-bit wire. This can be done with the `+&` summation, as seen on the cheatsheet.

```scala
val sum = io.in_a +& io.in_b
```

Finally, note that connecting a 4-bit `UInt` wire to a 5-bit `UInt` wire will truncate the MSB by default. You can use this to easily truncate the 5-bit sum for the non-saturating adder.

In [ ]:
class ParameterizedAdder(saturate: Boolean) extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val out  = Output(UInt(4.W))
  })

  ???
}

for (saturate <- Seq(true, false)) {
  test(new ParameterizedAdder(saturate)) { c =>
    // 100 random tests
    val cycles = 100
    import scala.util.Random
    import scala.math.min
    for (i <- 0 until cycles) {
      val in_a = Random.nextInt(16)
      val in_b = Random.nextInt(16)
      c.io.in_a.poke(in_a.U)
      c.io.in_b.poke(in_b.U)
      if (saturate) {
        c.io.out.expect(min(in_a + in_b, 15).U)
      } else {
        c.io.out.expect(((in_a + in_b) % 16).U)
      }
    }
    
    // ensure we test saturation vs. truncation
    c.io.in_a.poke(15.U)
    c.io.in_b.poke(15.U)
    if (saturate) {
      c.io.out.expect(15.U)
    } else {
      c.io.out.expect(14.U)
    }
  }
}
println("SUCCESS!!")

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-3" />
<label for="check-3"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val sum = io.in_a +& io.in_b
  if (saturate) {
    io.out := Mux(sum > 15.U, 15.U, sum)
  } else {
    io.out := sum
  }
</pre></article></div></section></div>

---
# You're done!

[Return to the top.](#top)